In [1]:
%pip install -U langchain-community langgraph langchain-anthropic tavily-python langgraph-checkpoint-sqlite

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
os.environ["LANGCHAIN_TRACING_V2"] = "true"  # Required to enable tracing
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"  # Optional (defaults to this)
os.environ["LANGCHAIN_API_KEY"] = "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"
os.environ["LANGCHAIN_PROJECT"] = "Agent with LangChain"

Tavily is the search engine

In [3]:
import getpass
import os

os.environ["TAVILY_API_KEY"] = getpass.getpass()

In [4]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=2)
search_results = search.invoke("what is the weather in SF")
print(search_results)
# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

[{'title': 'Weather in San Francisco', 'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1746651924, 'localtime': '2025-05-07 14:05'}, 'current': {'last_updated_epoch': 1746651600, 'last_updated': '2025-05-07 14:00', 'temp_c': 14.3, 'temp_f': 57.7, 'is_day': 1, 'condition': {'text': 'Mist', 'icon': '//cdn.weatherapi.com/weather/64x64/day/143.png', 'code': 1030}, 'wind_mph': 12.1, 'wind_kph': 19.4, 'wind_degree': 249, 'wind_dir': 'WSW', 'pressure_mb': 1020.0, 'pressure_in': 30.12, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 82, 'cloud': 75, 'feelslike_c': 12.8, 'feelslike_f': 55.1, 'windchill_c': 11.3, 'windchill_f': 52.3, 'heatindex_c': 12.6, 'heatindex_f': 54.6, 'dewpoint_c': 10.7, 'dewpoint_f': 51.3, 'vis_km': 8.0, 'vis_miles': 4.0, 'uv': 8.2, 'gust_mph': 14.7, 'gust_kph': 23.7}}", 'score': 0.6600

Language Models

In [5]:
%pip install -qU "langchain[google-genai]"

Note: you may need to restart the kernel to use updated packages.


In [6]:
import getpass
import os

if not os.environ.get("GOOGLE_API_KEY"):
  os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")

from langchain.chat_models import init_chat_model

model = init_chat_model("gemini-2.0-flash", model_provider="google_genai")

In [7]:
from langchain_core.messages import HumanMessage

response = model.invoke([HumanMessage(content="hi!")])
response.content

'Hi there! How can I help you today?'

Now let's enable the model to do tool calling

In [8]:
model_with_tools = model.bind_tools(tools)

Let's create the agent

In [9]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

For now thr queries are stateless (won't remember previous interactions)

Without need of tool

In [10]:
response = agent_executor.invoke({"messages": [HumanMessage(content="hi!")]})

response["messages"]

[HumanMessage(content='hi!', additional_kwargs={}, response_metadata={}, id='9ae7336f-ba53-42bc-a5c2-11d8fc5112d2'),
 AIMessage(content='Hi there! How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--8c7ec597-4bec-4d97-a488-a98dea6683ea-0', usage_metadata={'input_tokens': 51, 'output_tokens': 11, 'total_tokens': 62, 'input_token_details': {'cache_read': 0}})]

With tool

In [11]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}
)
response["messages"]

[HumanMessage(content='whats the weather in sf?', additional_kwargs={}, response_metadata={}, id='2203a311-7c87-4c33-8ea1-e908cf4dfa5f'),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'tavily_search_results_json', 'arguments': '{"query": "weather in San Francisco"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--9f1ee47c-b49f-449e-9594-8fdf062eac7b-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in San Francisco'}, 'id': '91d69168-409c-4808-ad2a-542613d24daa', 'type': 'tool_call'}], usage_metadata={'input_tokens': 55, 'output_tokens': 13, 'total_tokens': 68, 'input_token_details': {'cache_read': 0}}),
 ToolMessage(content='[{"title": "San Francisco weather in July 2025 | Weather25.com", "url": "https://www.weather25.com/north-america/usa/california/san-francisco?page=month&month=July", "content": "San Fra

Till now we have seen only the final response using .invoke
Instead streams allows to show the multiple steps the agent makes

In [12]:
for step in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather in sf?")]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

whats the weather in sf?
================================== Ai Message ==================================
Tool Calls:
  tavily_search_results_json (4f8fe77f-021e-40fa-88da-b9e1a91492d3)
 Call ID: 4f8fe77f-021e-40fa-88da-b9e1a91492d3
  Args:
    query: weather in San Francisco
================================= Tool Message =================================
Name: tavily_search_results_json

[{"title": "San Francisco weather in July 2025 | Weather25.com", "url": "https://www.weather25.com/north-america/usa/california/san-francisco?page=month&month=July", "content": "San Francisco weather in July 2025\n\nThe temperatures in San Francisco in July are comfortable with low of 14°C and and high up to 25°C.\n\nThere is little to no rain in San Francisco during July, so it’s a lot easier to explore the city. Just remember to dress in warm layers, as it can still get pretty chilly.\n\nOur weather forecast can give y

Now letìs try to give a memory to the agent using a MemorySaver and a thread_id

In [20]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}

for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im bob!")]}, config
):
    print(chunk)
    print("----")


{'agent': {'messages': [AIMessage(content='Hello Bob! How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--2fa21570-685d-4580-9357-c7d369a44e44-0', usage_metadata={'input_tokens': 53, 'output_tokens': 11, 'total_tokens': 64, 'input_token_details': {'cache_read': 0}})]}}
----


In [21]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Your name is Bob.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--86f823ee-4725-4c85-8934-346b962f5d4e-0', usage_metadata={'input_tokens': 67, 'output_tokens': 6, 'total_tokens': 73, 'input_token_details': {'cache_read': 0}})]}}
----


change thread to start a new conversation

In [22]:
config = {"configurable": {"thread_id": "xyz123"}}
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='I do not have access to personal information like your name. I am a language model, not a person.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--ec452d6d-a4a0-4c6e-a6d4-7650edd69164-0', usage_metadata={'input_tokens': 53, 'output_tokens': 23, 'total_tokens': 76, 'input_token_details': {'cache_read': 0}})]}}
----
